convert qupath annotation to imagescope xml in order to train deeplab model in MATLAB
210421 last edit

In [8]:
from paquo.projects import QuPathProject
import numpy as np
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom
import cv2
from matplotlib import pyplot as plt
from readxml_b import *
from time import time

In [9]:
s1 = time()
qppath = r"\\motherserverdw\Kyu_Sync\Research\Active\Aging_organized\svs\svs_back\qupath_v2\project.qpproj"
# with QuPathProject(qppath, mode='r+') as qp:
#     if os.path.exists(qp.images[0].uri[7:]):
#         print('loading project')
#     else:
#         print('updating uri')
#         if os.path.exists(r"\\motherserverdw\Kyu_Sync"):
#             for image in qp.images:
#                 qp.update_image_paths(uri2uri={image.uri: image.uri.replace('kukissd','10.162.80.6')})
#         elif os.path.exists(r"\\kukissd\Kyu_Sync"):
#             for image in qp.images:
#                 qp.update_image_paths(uri2uri={image.uri: image.uri.replace('10.162.80.6','kukissd')})
#         else:
#             raise NameError('server is not accessible')
qp=QuPathProject(qppath, mode='r')

print('quproj opened in {:.2f} sec'.format(time() - s1))

quproj opened in 0.06 sec


In [10]:
classnames = ['basale', 'bloodvessel', 'collagen', 'fat', 'granulosum', 'hairfollicle',
 'hairroot', 'inner corneum', 'nerve', 'oil', 'outer corneum', 'roi',
 'smoothmuscle', 'spinosum', 'sweat', 'white']
annsrc = r'\\motherserverdw\Kyu_Sync\Research\Active\Aging_organized\svs\class14_annotation_v2'
s2 = time()
for idx,image in enumerate(qp.images[21:]):
    start=time()
    print(idx,image.image_name)
    annpth = os.path.join(annsrc,image.image_name.replace('svs','xml'))
    if os.path.exists(annpth):
        s3 = time()
        coords,names = readxml_b(annpth)
        print('12 class annotation loaded in {:.2f} sec'.format(time() - s3))
    else:
        coords = []
        names = []
        print('12 class annotation does not exist for this image')
    #read annotation
    s4 = time()
    annotations = image.hierarchy.annotations
    if len(annotations)<1: continue
    for annotation in annotations:
        if not annotation.path_class is None:
            if annotation.path_class.name != "inflammation":
                if annotation.path_class.name != "noise":
                    poly = annotation.roi
                    if poly.type=='MultiPolygon':
                        # coords.append(np.concatenate(tuple([np.array(x.exterior.coords) for x in poly.geoms])).astype(np.int32))
                        dilation_count = 0
                        while poly.type=='MultiPolygon':
                            poly = poly.buffer(1)
                            if dilation_count>10: break
                            poly = poly.buffer(-1)
                            dilation_count += 1
                        # np.append(coords,np.array(poly.exterior.coords).astype(np.int32))
                        # np.append(names,annotation.path_class.name)
                        if not poly.type=='MultiPolygon':
                            coords.append(np.array(poly.exterior.coords).astype(np.int32))
                            names.append(annotation.path_class.name)
                    if poly.type=='Polygon':
                        # np.append(coords,np.array(poly.exterior.coords).astype(np.int32))
                        # np.append(names,annotation.path_class.name)
                        coords.append(np.array(poly.exterior.coords).astype(np.int32))
                        names.append(annotation.path_class.name)
    coords=np.array(coords)
    names =np.array(names)
    print('qupath annotation loaded in {:.2f} sec'.format(time() - s4))
    print(np.unique(names),len(np.unique(names)))

    if not set(np.unique(names)).issubset(set(classnames)):
        raise ValueError('check spelling for classes in either imagescope or qupath')

    s5=time()
    root = ET.Element("Annotations", MicronsPerPixel="0.230500")

    for idx,elem in enumerate(classnames):
        #basic setup
        m1 = ET.Element("Annotation", Id=str(idx+1), Name="", ReadOnly="0",
                        NameReadOnly="0", LineColorReadOnly="0",
                        Incremental="0", Type="4", LineColor="16711935",
                        Visible="1", Selected="0", MarkupImagePath="", MacroName="")
        root.append(m1)
        b1 = ET.SubElement(m1, "Attributes")
        c1 = ET.SubElement(b1, "Attribute", Name=elem, Id="0", Value="")
        b2 = ET.SubElement(m1, "Regions")
        c2 = ET.SubElement(b2, "RegionAttributeHeaders")
        ET.SubElement(c2, "AttributeHeader", Id="9999", Name="Region", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="9997", Name="Length", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="9996", Name="Area", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="9998", Name="Text", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="1", Name="Description", ColumnWidth="-1")
        # individual annotation circle
        for annidx,annelem in enumerate(coords[names==elem]):
            d2 = ET.SubElement(b2, "Region", Id=str(annidx+1), Type="0", Zoom="1", Selected="0", ImageLocation="", ImageFocus="-1",
            Text="", NegativeROA="0", InputRegionId="0", Analyze="1",DisplayId=str(annidx+1))
            ET.SubElement(d2,"Attributes")
            e2 = ET.SubElement(d2,"Vertices")
            for vertidx,vertelem in enumerate(annelem):
                ET.SubElement(e2,"Vertex", X=str(vertelem[0]), Y=str(vertelem[1]), Z="0")
        b3 = ET.SubElement(m1, "Plots")

    # xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")

    print('combined xml generated in {:.2f} sec'.format(time() - s5))
    print(image.image_name,'time elapsed {:.2f} sec'.format(time() - start))

    # with open(image.image_name.replace('svs','xml'),'wb') as ff:
    #     ff.write(xmlstr.encode('utf-8'))
    #
    with open(image.image_name.replace('svs','xml'),'wb') as f:
        ET.ElementTree(root).write(f)
    print(image.image_name.replace('svs','xml'),' file saved')
print(len(qp.images),'files processed in {:.2f} sec'.format(time() - s2))

0 wf61_b4_01.svs
12 class annotation does not exist for this image
1 wf63_b4_01.svs
12 class annotation does not exist for this image


<ipython-input-10-13aef9c2d538>:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  coords=np.array(coords)


qupath annotation loaded in 1.04 sec
['basale' 'bloodvessel' 'collagen' 'granulosum' 'inner corneum' 'roi'
 'spinosum' 'white'] 8
combined xml generated in 0.00 sec
wf63_b4_01.svs time elapsed 1.04 sec
wf63_b4_01.xml  file saved
2 wf64_b3_01.svs


C:\Users\kyuha\PycharmProjects\digital_pathology\skin_fibroblast\util\readxml_b.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  coords = np.array(coords)


12 class annotation loaded in 2.12 sec
qupath annotation loaded in 0.68 sec
['bloodvessel' 'collagen' 'fat' 'hairfollicle' 'hairroot' 'nerve' 'oil'
 'roi' 'smoothmuscle' 'sweat' 'white'] 11
combined xml generated in 0.50 sec
wf64_b3_01.svs time elapsed 3.30 sec
wf64_b3_01.xml  file saved
3 wf64_b4_01.svs
12 class annotation does not exist for this image
4 wf65_b4_01.svs
12 class annotation does not exist for this image
5 wf69_b3_01.svs
12 class annotation loaded in 0.13 sec
qupath annotation loaded in 1.24 sec
['basale' 'bloodvessel' 'collagen' 'fat' 'granulosum' 'hairfollicle'
 'hairroot' 'inner corneum' 'nerve' 'oil' 'outer corneum' 'roi'
 'smoothmuscle' 'spinosum' 'sweat' 'white'] 16
combined xml generated in 0.05 sec
wf69_b3_01.svs time elapsed 1.45 sec
wf69_b3_01.xml  file saved
6 wf73_b3_01.svs
12 class annotation loaded in 0.14 sec
qupath annotation loaded in 0.77 sec
['basale' 'bloodvessel' 'collagen' 'fat' 'granulosum' 'hairfollicle'
 'hairroot' 'inner corneum' 'nerve' 'outer 

TypeError: ('check name:', 'wm21_b2_01.xml', 'classes in file:', ['corneum', 'basale', 'hairroot', 'hairfollicle', 'smoothmuscle', 'oil', 'sweat', 'nerve', 'bloodvessel', 'collagen', 'fat', 'white'])